In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from ast import literal_eval
from statistics import median
import statistics
import math
pd.set_option('display.max_columns', None)
from matplotlib import pyplot
import seaborn as sn
from scipy.stats import pearsonr
import numpy as np
from scipy import stats as s
import lightgbm as lgb  # import LightGBM
import catboost as cb

In [4]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
# Load files into a pandas dataframe
train = pd.read_csv(r'C:\Solar Credit Repayment\Zipped_Solar\featured_train.csv')
test = pd.read_csv(r'C:\Solar Credit Repayment\Zipped_Solar\featured_test.csv')
ss = pd.read_csv('C:\Solar Credit Repayment\SampleSubmission.csv')
# metadata = pd.read_csv('C:\Solar Credit Repayment\metadata.csv')

In [6]:
train.head(2)

,Unnamed: 0,ID X Month,TransactionDates,PaymentsHistory,ID,target,Mean_PaymentsHistory,Median_PaymentsHistory,MostCommon_PaymentsHistory,variance_PaymentsHistory,stdev_PaymentsHistory,HighestPay_PaymentsHistory,LowestPay_PaymentsHistory,Sum_PaymentsHistory,Deposit,AccessoryRate,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,dayofweek_numRegistrationDate,quarterRegistrationDate,monthRegistrationDate,yearRegistrationDate,dayofyearRegistrationDate,dayofmonthRegistrationDate,weekRegistrationDate,SemesterRegistrationDate,DateRegistrationDate,is_weekendRegistrationDate,fortnightRegistrationDate,which_fortnightRegistrationDate,WeekOfyearRegistrationDate,HourofdayRegistrationDate,dayofweek_numExpectedTermDate,quarterExpectedTermDate,monthExpectedTermDate,yearExpectedTermDate,dayofyearExpectedTermDate,dayofmonthExpectedTermDate,weekExpectedTermDate,SemesterExpectedTermDate,DateExpectedTermDate,is_weekendExpectedTermDate,fortnightExpectedTermDate,which_fortnightExpectedTermDate,WeekOfyearExpectedTermDate,HourofdayExpectedTermDate,dayofweek_numFirstPaymentDate,quarterFirstPaymentDate,monthFirstPaymentDate,yearFirstPaymentDate,dayofyearFirstPaymentDate,dayofmonthFirstPaymentDate,weekFirstPaymentDate,SemesterFirstPaymentDate,DateFirstPaymentDate,is_weekendFirstPaymentDate,fortnightFirstPaymentDate,which_fortnightFirstPaymentDate,WeekOfyearFirstPaymentDate,HourofdayFirstPaymentDate,dayofweek_numLastPaymentDate,quarterLastPaymentDate,monthLastPaymentDate,yearLastPaymentDate,dayofyearLastPaymentDate,dayofmonthLastPaymentDate,weekLastPaymentDate,SemesterLastPaymentDate,DateLastPaymentDate,is_weekendLastPaymentDate,fortnightLastPaymentDate,which_fortnightLastPaymentDate,WeekOfyearLastPaymentDate,HourofdayLastPaymentDate,rateTypeEntity_DAILY,rateTypeEntity_MONTHLY,rateTypeEntity_WEEKLY,MainApplicantGender_Female,MainApplicantGender_Male,Region_Coast Region,Region_Mount Kenya Region,Region_Nairobi Region,Region_North Rift,Region_Nyanza,Region_South Rift,Region_Western,Town_Baringo,Town_Bomet,Town_Bungoma,Town_Busia,Town_Elgeyo/Marakwet,Town_Embu,Town_Garissa,Town_Homa Bay,Town_Isiolo,Town_Kajiado,Town_Kakamega,Town_Kericho,Town_Kiambu,Town_Kilifi,Town_Kirinyaga,Town_Kisii,Town_Kisumu,Town_Kitui,Town_Kwale,Town_Laikipia,Town_Lamu,Town_Machakos,Town_Makueni,Town_Mandera,Town_Marsabit,Town_Meru,Town_Migori,Town_Mombasa,Town_Muranga,Town_Nairobi City,Town_Nakuru,Town_Nandi,Town_Narok,Town_Nyamira,Town_Nyandarua,Town_Nyeri,Town_Samburu,Town_Siaya,Town_Taita/Taveta,Town_Tana River,Town_Tharaka-Nithi,Town_Trans Nzoia,Town_Turkana,Town_UNKNOWN,Town_Uasin Gishu,Town_Vihiga,Town_Wajir,Town_West Pokot,Occupation_Business,Occupation_Driver/Motorbike Rider,Occupation_Farmer,Occupation_Government Employee,Occupation_Labourer,Occupation_Other,Occupation_Teacher,SupplierName_d_light,weekdayRegistrationDate_Friday,weekdayRegistrationDate_Monday,weekdayRegistrationDate_Saturday,weekdayRegistrationDate_Sunday,weekdayRegistrationDate_Thursday,weekdayRegistrationDate_Tuesday,weekdayRegistrationDate_Wednesday,is_month_startRegistrationDate_False,is_month_startRegistrationDate_True,is_month_endRegistrationDate_False,is_month_endRegistrationDate_True,is_quarter_startRegistrationDate_False,is_quarter_startRegistrationDate_True,is_quarter_endRegistrationDate_False,is_quarter_endRegistrationDate_True,is_year_startRegistrationDate_False,is_year_startRegistrationDate_True,is_year_endRegistrationDate_False,is_year_endRegistrationDate_True,weekdayExpectedTermDate_Friday,weekdayExpectedTermDate_Monday,weekdayExpectedTermDate_Saturday,weekdayExpectedTermDate_Sunday,weekdayExpectedTermDate_Thursday,weekdayExpectedTermDate_Tuesday,weekdayExpectedTermDate_Wednesday,is_month_startExpectedTermDate_False,is_month_startExpectedTermDate_True,is_month_endExpectedTermDate_False,is_month_endExpectedTermDate_True,is_quarter_startExpectedTermDate_False,is_quarter_startExpectedTermDate_True,is_quarter_endExpectedTermDate_False,is_quarter_endExpectedTermDate_True,is_year_startExpectedTe

In [7]:
test.head(2)

,Unnamed: 0,ID X Month,TransactionDates,PaymentsHistory,ID,Mean_PaymentsHistory,Median_PaymentsHistory,MostCommon_PaymentsHistory,variance_PaymentsHistory,stdev_PaymentsHistory,HighestPay_PaymentsHistory,LowestPay_PaymentsHistory,Sum_PaymentsHistory,Deposit,AccessoryRate,RatePerUnit,DaysOnDeposit,Age,Term,TotalContractValue,dayofweek_numRegistrationDate,quarterRegistrationDate,monthRegistrationDate,yearRegistrationDate,dayofyearRegistrationDate,dayofmonthRegistrationDate,weekRegistrationDate,SemesterRegistrationDate,DateRegistrationDate,is_weekendRegistrationDate,fortnightRegistrationDate,which_fortnightRegistrationDate,WeekOfyearRegistrationDate,HourofdayRegistrationDate,dayofweek_numExpectedTermDate,quarterExpectedTermDate,monthExpectedTermDate,yearExpectedTermDate,dayofyearExpectedTermDate,dayofmonthExpectedTermDate,weekExpectedTermDate,SemesterExpectedTermDate,DateExpectedTermDate,is_weekendExpectedTermDate,fortnightExpectedTermDate,which_fortnightExpectedTermDate,WeekOfyearExpectedTermDate,HourofdayExpectedTermDate,dayofweek_numFirstPaymentDate,quarterFirstPaymentDate,monthFirstPaymentDate,yearFirstPaymentDate,dayofyearFirstPaymentDate,dayofmonthFirstPaymentDate,weekFirstPaymentDate,SemesterFirstPaymentDate,DateFirstPaymentDate,is_weekendFirstPaymentDate,fortnightFirstPaymentDate,which_fortnightFirstPaymentDate,WeekOfyearFirstPaymentDate,HourofdayFirstPaymentDate,dayofweek_numLastPaymentDate,quarterLastPaymentDate,monthLastPaymentDate,yearLastPaymentDate,dayofyearLastPaymentDate,dayofmonthLastPaymentDate,weekLastPaymentDate,SemesterLastPaymentDate,DateLastPaymentDate,is_weekendLastPaymentDate,fortnightLastPaymentDate,which_fortnightLastPaymentDate,WeekOfyearLastPaymentDate,HourofdayLastPaymentDate,rateTypeEntity_DAILY,rateTypeEntity_MONTHLY,rateTypeEntity_WEEKLY,MainApplicantGender_Female,MainApplicantGender_Male,Region_Coast Region,Region_Mount Kenya Region,Region_Nairobi Region,Region_North Rift,Region_Nyanza,Region_South Rift,Region_Western,Town_Baringo,Town_Bomet,Town_Bungoma,Town_Busia,Town_Elgeyo/Marakwet,Town_Embu,Town_Garissa,Town_Homa Bay,Town_Isiolo,Town_Kajiado,Town_Kakamega,Town_Kericho,Town_Kiambu,Town_Kilifi,Town_Kirinyaga,Town_Kisii,Town_Kisumu,Town_Kitui,Town_Kwale,Town_Laikipia,Town_Lamu,Town_Machakos,Town_Makueni,Town_Mandera,Town_Marsabit,Town_Meru,Town_Migori,Town_Mombasa,Town_Muranga,Town_Nairobi City,Town_Nakuru,Town_Nandi,Town_Narok,Town_Nyamira,Town_Nyandarua,Town_Nyeri,Town_Samburu,Town_Siaya,Town_Taita/Taveta,Town_Tana River,Town_Tharaka-Nithi,Town_Trans Nzoia,Town_Turkana,Town_UNKNOWN,Town_Uasin Gishu,Town_Vihiga,Town_Wajir,Town_West Pokot,Occupation_Business,Occupation_Driver/Motorbike Rider,Occupation_Farmer,Occupation_Government Employee,Occupation_Labourer,Occupation_Other,Occupation_Teacher,SupplierName_d_light,weekdayRegistrationDate_Friday,weekdayRegistrationDate_Monday,weekdayRegistrationDate_Saturday,weekdayRegistrationDate_Sunday,weekdayRegistrationDate_Thursday,weekdayRegistrationDate_Tuesday,weekdayRegistrationDate_Wednesday,is_month_startRegistrationDate_False,is_month_startRegistrationDate_True,is_month_endRegistrationDate_False,is_month_endRegistrationDate_True,is_quarter_startRegistrationDate_False,is_quarter_startRegistrationDate_True,is_quarter_endRegistrationDate_False,is_quarter_endRegistrationDate_True,is_year_startRegistrationDate_False,is_year_startRegistrationDate_True,is_year_endRegistrationDate_False,is_year_endRegistrationDate_True,weekdayExpectedTermDate_Friday,weekdayExpectedTermDate_Monday,weekdayExpectedTermDate_Saturday,weekdayExpectedTermDate_Sunday,weekdayExpectedTermDate_Thursday,weekdayExpectedTermDate_Tuesday,weekdayExpectedTermDate_Wednesday,is_month_startExpectedTermDate_False,is_month_startExpectedTermDate_True,is_month_endExpectedTermDate_False,is_month_endExpectedTermDate_True,is_quarter_startExpectedTermDate_False,is_quarter_startExpectedTermDate_True,is_quarter_endExpectedTermDate_False,is_quarter_endExpectedTermDate_True,is_year_startExpectedTermDate_

## Remove Date, TransactionDates and PaymentsHistory from the Dataset


In [8]:
deleted = ['Unnamed: 0', 'ID X Month',	'TransactionDates', 'PaymentsHistory']	
for col in deleted:
    del train[col]
    del test[col]

## Find the Difference between each Payment Details

In [9]:
# bound = 6
# for i in range(168042):
#     if i = 6:
#     train.loc[i, 'target_payment'] - train.loc[i + 1, 'target_payment']


In [10]:
train.shape

(168042, 228)

In [11]:
# values_list = []
# for i in range(168042):
#     values_list.append(6)


In [22]:
## Define columns to be used for training
all_cols = [col for col in train.columns if col !='target' and col != 'ID']

## Modelling

In [24]:
train_data = lgb.Dataset(train[all_cols], train['target'],     
                     feature_name = all_cols)

# defining parameters
SEARCH_PARAMS = {'learning_rate': [0.05],
                'max_depth': [16],
                 'boosting_type': 'gbdt',
                'num_leaves': [600],
                'feature_fraction': 0.8,
                'subsample': 0.3}

In [25]:
model = lgb.train(SEARCH_PARAMS, train_data,             
                     num_boost_round=300)

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046380 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5727
[LightGBM] [Info] Number of data points in the train set: 168042, number of used features: 225
[LightGBM] [Info] Start training from score 792.500018
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [27]:
# Score the model on the train set
from sklearn.metrics import log_loss
log_loss(train['target'], model.predict_proba(train[all_cols])[:, 1])

AttributeError: 'Booster' object has no attribute 'predict_proba'

In [3]:
=

'1.20.3'

In [7]:
!pip install pandas==1.2.4